In [6]:
# https://stackoverflow.com/questions/59008756/getting-google-search-result-urls-from-search-string-or-url
import requests
from bs4 import BeautifulSoup
import pandas as pd

example_queries_url = "https://raw.githubusercontent.com/nhsx/data-lens-query-validator/main/example_queries.csv"
df = pd.read_csv(example_queries_url, error_bad_lines=False)
search_url = "http://www.google.com/search?q={}&hl=en"


query_results = []
for query_search_term, correct_result_URL in zip(
    df.query_search_term, df.correct_result_URL
):
    result_list = []
    search_request = requests.get(
        search_url.format(query_search_term.replace(" ", "+")),                 # add query string to search url and request
        headers={"User-Agent": "Mozilla/5.0"},
    )
    soup = BeautifulSoup(search_request.text, "lxml")                           # parse HTML
    for search_result in soup.findAll("a"):                                     # find all hypertext link tags
        result_list.append(search_result.get("href"))                           # append list with each href (Hypertext REFerence) url i.e., <a href="url">text<a/>
    if any(correct_result_URL in url for url in result_list):                   # find any url that matches the data url we are looking for
        query_results.append(correct_result_URL)
    else:
        query_results.append(None)

results_dict = dict(zip(df.query_search_term, query_results))
df_google = pd.DataFrame(results_dict.items(), columns=['Query', 'Result'])

df_google


,Query,Result
0,Patient Online data,https://digital.nhs.uk/data-and-information/da...
1,Electronic Prescription Service data,None
2,NHS 111 data,https://www.england.nhs.uk/statistics/statisti...
3,Reported Covid symptoms data,None
4,GP Appointments data,None
5,Number of patients registered at GP practice data,https://digital.nhs.uk/data-and-information/pu...
6,CCG e-Referral data,None
7,e-Referral data,None
8,Digital Maturity Assessment,None
